### Split-Step Fourier to solve the Time-Dependent Schrödinger Equation

The evolution of the Quantum State $\psi(x, t)$ is governed by the Schrödinger Equation:

$$
    i\hbar \frac{\partial}{\partial t} \psi(x, t) = \hat{H}\psi(x, t)
$$

Where the Hamiltonian $\hat{H}$ is the sum of Kinetic ($\hat{T}$) and Potential ($\hat{V}$) Operators:

$$
    \hat{H} = -\frac{\hbar^{2}}{2m} \frac{\partial^{2}}{\partial x^{2}} + V(x)
$$

For this simulation, we work in Atomic Units where $\hbar = 1$ and mass $m = 1$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import warnings
np.seterr(over='ignore', under='ignore', invalid='ignore', divide='ignore')
warnings.filterwarnings('ignore')

'''
Configuration
'''
# Physical Length of the universe 
L = 100.0
# Spacial Resolution (Power of 2 for optimal FFT)
N = 1024
# Time Step
dt = 0.05
# Total time steps
t_max = 200

To solve this efficiently, we must exist in two places at once:
1. Position Space ($x$): Where Potential $V(x)$ is defined.
2. Momentum Space ($k$): Where Kinetic energy $T(k)$ is defined.

<br>

The Discrete Fourier Transform (DFT) links these two domains:

$$
x_n = -L/2 + n \Delta x \quad \xrightarrow{\mathcal{F}} \quad k_n = \frac{2\pi n}{L}
$$

In [2]:
# 1. Position Space Grid 
x = np.linspace(-L/2, L/2, N)
dx = x[1] - x[0]

# 2. Momentum Space Grid
k = 2 * np.pi * np.fft.fftfreq(N, d=dx)

The formal solution to the wave equation is $\psi(t + \Delta t) = e^{-i\hat{H}\Delta t}\psi (t)$. 


Since $\hat{T}$ (momentum) and $\hat{V}$ (position) do not commute ($[\hat{x}, \hat{p}] \neq 0$), we cannot simply write $e^{-i(\hat{T} + \hat{V})t} = e^{-i\hat{T}t}e^{-i\hat{V}t}$.


We use **Strang Splitting** to approximate the evolution unitary operator with $O(\Delta t^{3})$ error:

$$
e^{-i(\hat{T} + \hat{V}t)} \approx e^{-i\hat{V} \frac{\Delta t}{2}} \cdot e^{-i\hat{T} \Delta t} \cdot e^{-i\hat{V} \frac{\Delta t}{2}}
$$

This dictates our algorithmic step:
1. Half-Kick: Apply half the potential in $x$-space.
1. Drift: Apply full kinetic energy in $k$-space.
1. Half-Kick: Apply half the potential in $x$-space.

In [3]:
class QuantumSolver:
    def __init__(self, x, k, V, dt):
        self.x = x
        self.k = k
        self.dt = dt
        self.N = len(x)
        self.psi = np.zeros(self.N, dtype=np.complex128)

        # precompute kinetic operator in k-space
        # T = p ^ 2 / 2m = k ^ 2 / 2
        self.T_k = 0.5 * (k ** 2)
        self.U_kin = np.exp(-1j * self.T_k * dt)

        # precompute potential operator in x-space
        self.U_pot_half = np.exp(-1j * V * dt / 2)

    def set_wave_fn(self, psi_0):
        self.psi = psi_0.astype(np.complex128)

    def step(self):
        # half kick 
        # psi <- exp(-i * V * dt/2) * psi
        np.multiply(self.psi, self.U_pot_half, out=self.psi)
        
        # Drift
        # transform to k-space
        psi_k = np.fft.fft(self.psi)

        # kinetic propogator
        # psi_k <- exp(-i * T * dt) * psi_k 
        np.multiply(psi_k, self.U_kin, out=psi_k)

        # transoform back to x-space
        self.psi = np.fft.ifft(psi_k)

        # half kick 
        # psi <- exp(-i * V * dt/2) * psi
        np.multiply(self.psi, self.U_pot_half, out=self.psi)

We create a Gaussian wave packet representing a particle moving to the right, and place a rectangular potential barrier in its path.

The Wave Packet:

$$
\psi_0(x) = \frac{1}{{(\pi \sigma ^ {2})}^{1/4}} e ^ {-\frac{(x - x_0)^2}{2\sigma ^ 2}} e^{ik_0x}
$$

The Barrier:

$$
V(x) = \begin{cases} V_0 & \text{if } x \in [10, 12] \\ 0 & \text{otherwise} \end{cases}
$$

In [4]:
# Define Potential V(x)
V_x = np.zeros(N)

# Make a barrier of width 2 and height 1.5
mask = (x > 10) & (x < 12)
V_x[mask] = 1.5

# Instantiate solver
solver = QuantumSolver(x, k, V_x, dt)

'''
Define initial wave function (Gaussian)
'''
# start position
x0 = -20.0 
# Initial momentum (velocity)
k0 = 1.5
# Packet width 
sigma = 2.0

norm = (1 / (np.pi * (sigma ** 2)))**(1/4) 

psi_0 = norm * np.exp(-(x - x0)**2 / (2 * sigma**2)) * np.exp(1j * k0 * x)

solver.set_wave_fn(psi_0)

In [5]:
solver.set_wave_fn(psi_0)

fig, ax = plt.subplots(figsize=(10, 6))
fig.patch.set_facecolor('black')
ax.set_facecolor('black')

ax.set_title('Quantum Tunneling via Split-Step Fourier Method', color='white', fontsize=16, pad=20)
ax.set_xlabel("Position ($x$)", color='white', fontsize=12)
ax.set_ylabel("Probability Density ($|\Psi(x,t)|^2$)", color='white', fontsize=12)

ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
for spine in ax.spines.values():
    spine.set_edgecolor('white')

ax.plot(x, V_x, 'r-', lw=2, alpha=0.6, label='Potential V(x)')
ax.fill_between(x, V_x, color='red', alpha=0.1)

line, = ax.plot(x, np.abs(psi_0)**2, 'c-', lw=2.5, label=r'$|\Psi(x, t)|^2$')

ax.set_ylim(0, 0.25)
ax.set_xlim(-50, 50)
ax.grid(True, color='gray', linestyle='--', alpha=0.3)
legend = ax.legend(loc="upper right", framealpha=0.2)
plt.setp(legend.get_texts(), color='white')

def animate(i):
    for _ in range(5):
        solver.step()
    
    prob_density = np.abs(solver.psi) ** 2
    line.set_data(x, prob_density)
    return line,

print("Compiling animation... (This may take a minute)")
ani = animation.FuncAnimation(fig, animate, frames=200, interval=30, blit=False)

ani.save(
    'quantum_tunneling.mp4', 
    writer='ffmpeg', 
    fps=60,            
    dpi=150,           
    bitrate=3000,      
    codec='h264',      
    extra_args=['-preset', 'slow', '-crf', '18'] # FFMPEG flags for maximum quality
)

print("Saved high-quality render to 'quantum_tunneling.mp4'")
plt.close() 

Compiling animation... (This may take a minute)
Saved high-quality render to 'quantum_tunneling.mp4'
